In [22]:

def is_power_of_two(n : int) -> bool:
    if n == 0:
        return False
    return (n & (n - 1)) == 0

def is_fermat_power(n : int) -> bool:
    if n < 4_000_000_000:
        return (n == 2) | (n == 4) | (n == 16) | (n == 256) | (n == 65536)
    elif not is_power_of_two(n):
        return False
    else:
        return is_power_of_two(n.bit_length() - 1)

def nim_sum(n : int,m : int) -> int:
    return n ^ m

def nim_prod(x : int,y : int) -> int:
    # first handle trivial cases
    if x == 0 | y == 0:
        return 0
    elif x == 1:
        return y
    elif y == 1:
        return x
    
    # next, use the rule for multiplying Fermat powers F=2 ** (2 ** n)
    # if x < F, then nim_prod(x,F)= x*F   (ordinary product)
    # and nim_prod(F,F) = 3*F/2
    m = min(x,y)
    M = max(x,y)
    
    if is_fermat_power(M):
        # nim product of Fermat power with smaller is ordinary product
        if m < M:
            return m*M
        else:
            # nim square of fermat power x is 3x/2
            return 3*M >> 1
    elif is_power_of_two(M):
        # if exponent is not power of 2, factor out 2's until it is
        # M = (factored) * (2 ** exponent)
        # we know at least one 2 needs to be pulled out        
        exponent = 1
        factored = M >> 1
        while not is_fermat_power(factored):
            factored >>= 1
            exponent += 1
        # now use formula for fermat power and associativity
        # m* M = (m * factored) * (2 ** exponent) = (m * (2 ** exponent)) * factored
        # we have to re-order the parentheses carefully to avoid infinite loop
        intermediate = nim_prod(m, factored)
        if intermediate == M:
            intermediate = nim_prod(m, 1 << exponent)
            return nim_prod(intermediate, factored)
        return nim_prod(intermediate, 1 << exponent)
    else:
        # otherwise, write it as the sum of powers of 2 and distribute
        sum = 0
        for index in range(M.bit_length()):
            if M >> index & 1 == 1:
                sum ^= nim_prod(m, 1 << index)
        return sum 
    
def nim_power(x : int,n : int) -> int:
    if n == 0:
        return 1
    elif n == 1:
        return x
    elif x == 0:
        return 0
    elif x == 1:
        return 1
    else:
        return nim_prod(nim_power(x,n-1),x)
    
def nim_order(n : int) -> int:
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        order = 1
        while nim_power(n,order) != 1:
            order += 1
        return order


In [12]:
def least_fermat(N : int) -> int:
    '''
    Find the least Fermat power <= N 
    2 ** (2 ** n) <= N
    '''
    if N < 2:
        raise ValueError('The lowest Fermat power is 2')
    bit_log = N.bit_length() - 1
    bit_log_log = bit_log.bit_length()-1
    return 1 << (1 << bit_log_log)

def nim_product(a : int, b : int) -> int:
    # first handle trivial cases
    if a == 0 or b == 0:
        return 0
    elif a == 1:
        return b
    elif b == 1:
        return a
    elif a == 2 and b == 2:
        return 3
    else:
        # do euclidean division by greatest possible fermat power 
        # a = q_a * F_a + r_a and b = q_b * F_b + r_b
        F_a = least_fermat(a); F_b = least_fermat(b)
        q_a = a // F_a ; q_b = b // F_b
        r_a = a % F_a ; r_b = b % F_b

        # if one the Fermat powers is greater than the other, then
        # nim multiplication by it is the same as ordinary multiplication
        if F_a < F_b:
            return nim_product(a,q_b)*F_b ^ nim_product(a,r_b)
        elif F_a > F_b:
            return nim_product(q_a,b)*F_a ^ nim_product(r_a,b)
        else:
            # otherwise we have to distribute and use F_n ** 2 = 3 * F_n / 2
            p_1 = nim_product(q_a,q_b)
            p_2 = nim_product(r_a,r_b)
            p_3 = nim_product(q_a ^ r_a, q_b ^ r_b)
            p_4 = nim_product(p_1, F_a >> 1)
            p_5 = p_3 ^ p_2
            return p_5 * F_a ^ p_2 ^ p_4



In [16]:
38% 16


6

In [16]:
import numpy as np
import sympy as sp

M = np.zeros((16,16),dtype=int)
for x in range(16):
    for y in range(16):
        M[x,y] = nim_product(x,y)

sp.Matrix(M)


Matrix([
[0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
[0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
[0,  2,  3,  1,  8, 10, 11,  9, 12, 14, 15, 13,  4,  6,  7,  5],
[0,  3,  1,  2, 12, 15, 13, 14,  4,  7,  5,  6,  8, 11,  9, 10],
[0,  4,  8, 12,  6,  2, 14, 10, 11, 15,  3,  7, 13,  9,  5,  1],
[0,  5, 10, 15,  2,  7,  8, 13,  3,  6,  9, 12,  1,  4, 11, 14],
[0,  6, 11, 13, 14,  8,  5,  3,  7,  1, 12, 10,  9, 15,  2,  4],
[0,  7,  9, 14, 10, 13,  3,  4, 15,  8,  6,  1,  5,  2, 12, 11],
[0,  8, 12,  4, 11,  3,  7, 15, 13,  5,  1,  9,  6, 14, 10,  2],
[0,  9, 14,  7, 15,  6,  1,  8,  5, 12, 11,  2, 10,  3,  4, 13],
[0, 10, 15,  5,  3,  9, 12,  6,  1, 11, 14,  4,  2,  8, 13,  7],
[0, 11, 13,  6,  7, 12, 10,  1,  9,  2,  4, 15, 14,  5,  3,  8],
[0, 12,  4,  8, 13,  1,  9,  5,  6, 10,  2, 14, 11,  7, 15,  3],
[0, 13,  6, 11,  9,  4, 15,  2, 14,  3,  8,  5,  7, 10,  1, 12],
[0, 14,  7,  9,  5, 11,  2, 12, 10,  4, 13,  3, 15,  1,  8,  6],
[0, 15,  5, 10, 

In [17]:
M = np.zeros((1,255),dtype=int)
for x in range(1,256):
    print(f'nim_({x}) = {nim_order(x)}' )

sp.Matrix(M)

nim_order(1) = 0
nim_order(2) = 3
nim_order(3) = 3
nim_order(4) = 15
nim_order(5) = 15
nim_order(6) = 15
nim_order(7) = 15
nim_order(8) = 5
nim_order(9) = 15
nim_order(10) = 5
nim_order(11) = 15
nim_order(12) = 15
nim_order(13) = 5
nim_order(14) = 5
nim_order(15) = 15
nim_order(16) = 85
nim_order(17) = 85


RecursionError: maximum recursion depth exceeded

In [21]:
bin(133)

'0b10000101'